In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sell_reduced.csv')
# Drop zeros
df = df[df['usd price'] != 0]
df = df.dropna()
df = df.drop_duplicates()
df.head()

,id,date,property_type,lat,lon,local price,usd price,barrio,commune
0,155a59477c557a77de931cb2ef91d6a9b23019d7,8/8/2014,apartment,-34.633020,-58.375759,3109410.0,180000.0,32,4
1,a9ba8add5599612a2129460b3c4080da2d1d1e67,8/8/2014,apartment,-34.632666,-58.446940,3109410.0,180000.0,9,7
2,dea72061724399f22c81b42424a93102ba42ea47,12/4/2014,apartment,-34.558970,-58.457864,3109410.0,180000.0,1,13
3,dc101767062be47bb2e2153d385b8841c7187e36,9/16/2014,apartment,-34.565875,-58.463096,3109410.0,180000.0,1,13
4,e683af5e8ceb6c28e7b067e5339e136d321f4f33,9/8/2014,apartment,-34.570216,-58.448173,3109410.0,180000.0,28,13


In [3]:
df.describe()

,lat,lon,local price,usd price,barrio,commune
count,179690.000000,179690.000000,1.796900e+05,1.796900e+05,179690.000000,179690.000000
mean,-34.600507,-58.434548,4.358570e+06,2.558001e+05,18.622110,8.094107
std,0.026431,0.039863,7.604861e+06,4.444552e+05,15.923544,5.015090
min,-34.699397,-58.531170,4.757708e+04,2.754180e+03,0.000000,1.000000
25%,-34.620132,-58.464089,1.676692e+06,9.900000e+04,4.000000,3.000000
50%,-34.601380,-58.433454,2.591175e+06,1.500000e+05,14.000000,8.000000
75%,-34.581850,-58.403132,4.571050e+06,2.694188e+05,35.000000,13.000000
max,-34.534315,-58.344687,8.212711e+08,4.654544e+07,48.000000,15.000000


In [4]:
df.date = pd.to_datetime(df.date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179690 entries, 0 to 360422
Data columns (total 9 columns):
id               179690 non-null object
date             179690 non-null datetime64[ns]
property_type    179690 non-null object
lat              179690 non-null float64
lon              179690 non-null float64
local price      179690 non-null float64
usd price        179690 non-null float64
barrio           179690 non-null int64
commune          179690 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 13.7+ MB


In [5]:
lower_class = np.percentile(df['usd price'], 10)
print(lower_class)

75000.0


In [6]:
upper_class = np.percentile(df['usd price'], 90)
print(upper_class)

495000.0


In [7]:
r_df = df[['barrio', 'date', 'local price', 'usd price']]

aggregation = {
    'usd price': {
        'count': 'count',
        'mean': 'mean',
        'count 10th percentile': lambda x: (x <= lower_class).sum(),
        'count 90th percentile': lambda x: (x >= upper_class).sum(),
        'min': 'min',
        'max': 'max',
        'std': 'std'
    }
}

stats = r_df.groupby(['barrio']).agg(aggregation)

/home/rocket/miniconda3/envs/casas/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
stats = pd.DataFrame(stats['usd price'][['count', 'mean', 'count 10th percentile', 'count 90th percentile', 'min', 'max', 'std']])

In [9]:
stats.head()

,count,mean,count 10th percentile,count 90th percentile,min,max,std
barrio,,,,,,,
0,22594,391356.869000,446.0,4697.0,5354.16,15692300.0,539280.606697
1,10580,363250.780320,222.0,1991.0,6078.54,5500000.0,465556.598866
2,4296,299767.185994,94.0,472.0,4851.34,6000000.0,413674.751398
3,1470,253757.316687,83.0,125.0,46817.00,6500000.0,433544.748000
4,9551,161716.585519,1727.0,260.0,6985.60,5000000.0,205693.826657


In [10]:
stats.to_csv('barrio_stats.csv')

In [11]:
stats = r_df.groupby(['barrio', pd.Grouper(key="date", freq='M')]).agg(aggregation)

In [12]:
stats = pd.DataFrame(stats['usd price'][['count', 'mean', 'count 10th percentile', 'count 90th percentile', 'min', 'max', 'std']])

In [13]:
stats.head()

count           mean  count 10th percentile  \
barrio date                                                      
0      2014-07-31     37  245822.270270                    1.0   
       2014-08-31     47  269677.580213                    2.0   
       2014-09-30     62  286945.967742                    4.0   
       2014-10-31     34  264371.738824                    1.0   
       2014-11-30     98  334166.788673                    4.0   

                   count 90th percentile      min        max            std  
barrio date                                                                  
0      2014-07-31                    4.0  70000.0  1200000.0  249421.914162  
       2014-08-31                    7.0  70000.0  1500000.0  318631.889964  
       2014-09-30                   12.0  50000.0  1200000.0  271109.899121  
       2014-10-31                    4.0  50000.0  1700000.0  293474.181266  
       2014-11-30                   22.0  58000.0  1380000.0  303975.122635

In [14]:
stats.to_csv('barrio_month_stats.csv')